In [1]:
import glob
import random

In [5]:
files = glob.glob("../ArtNet_Data/Annotations/*.xml")
all_images =[]
train = []
val =[]
test =[] 
for file in files:
    name = file.split('/')[3].split('.')[0]
    all_images.append(name)
    rand_number = random.random()
    if(rand_number <0.7): 
        train.append(name)
    elif(rand_number<0.9):
        val.append(name)
    else:
        test.append(name)

def Write_Data(file, fileNames):   
    text_file = open("../ArtNet_Data/ImageSets/Main/"+file, "w+")
    for name in fileNames:
        text_file.write(name + '\n')
    text_file.close()

Write_Data('train.txt', train)
Write_Data('trainval.txt', val)
Write_Data('val.txt', val)
Write_Data('test.txt', test)

len(all_images), len(train), len(test), len(val)
    

(345, 238, 30, 77)